## 导入库

In [42]:
import tensorflow as tf
import numpy as np
import pandas as pd
import functools

## 加载数据

### 读取训练数据和测试数据路径

In [20]:
train_file_path = ''
test_file_path = ''

### 设置标签列

In [25]:
LABEL_COLUMN = 'PM'

### 构建tensor数据集格式

In [26]:
def get_dataset(file_path):
    """
    构建tensorflow的数据集格式
    """
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=12,
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True)
    return dataset

In [30]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

## 数据预处理

### 连续数据

In [37]:
def process_continuous_data(mean, data):
  # 标准化数据
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

In [63]:
MEANS = {
    'TOAb1': 0.377115818,
    'TOAb2': 0.327171313,
    'TOAb3': 0.274613273,
    'TOAb4': 0.335810444,
    'TOAb5': 0.253371053,
    'TOAb6': 0.172047367,
    'SAA': 139.8266326,
    'SAZ': 51.14860631,
    'SOA': 4.170314475,
    'SOZ': 64.99057161,
    'V10': -2.188058381,
    'U10': -1.117582186,
    'BLH': 2591.388364,
    'ET': -0.000456378,
    'SP': 157312.8404,
    'PRE': 0.006855383,
    'TEM': 1.713888597,
    'RH': 55.03757061,
    'DEM': 0.462943548,
    'LUCC': 12.05407903,
    'NDVI': 0.104417258
}


numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

### 创建预处理层

In [65]:
preprocessing_layer = tf.keras.layers.DenseFeatures(numerical_columns)

## 搭建网络

In [66]:
# 构建一个DNN模型h(g(f(x)))
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

## 配置训练方法

In [67]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

## 执行训练过程

In [68]:
train_data = raw_train_data.shuffle(500)
test_data = raw_test_data

model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
6855/6855 [==============================] - 10s 1ms/step - loss: -8209461033.8386 - accuracy: 8.0032e-04
Epoch 2/20
6855/6855 [==============================] - 9s 1ms/step - loss: -314067124039.9907 - accuracy: 7.0997e-04
Epoch 3/20
6855/6855 [==============================] - 10s 1ms/step - loss: -1883958101688.6067 - accuracy: 7.5661e-04
Epoch 4/20
6855/6855 [==============================] - 11s 1ms/step - loss: -6199247565378.3154 - accuracy: 7.0255e-04
Epoch 5/20
6855/6855 [==============================] - 11s 2ms/step - loss: -15202297895746.0156 - accuracy: 8.6092e-04
Epoch 6/20
6855/6855 [==============================] - 10s 1ms/step - loss: -31216160301180.2656 - accuracy: 7.7984e-04
Epoch 7/20
6855/6855 [==============================] - 10s 1ms/step - loss: -56896738855578.7344 - accuracy: 8.0201e-04
Epoch 8/20
6855/6855 [==============================

In [69]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
2227/2227 [==============================] - 3s 1ms/step - loss: -3204573346922496.0000 - accuracy: 3.7420e-04


Test Loss -3204573346922496.0, Test Accuracy 0.0003741954860743135


In [70]:
predictions = model.predict(test_data)

# 显示部分结果
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

Consider rewriting this model with the Functional API.
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED
Predicted survival: 100.00%  | Actual outcome:  SURVIVED


## 打印网络结构

In [ ]:
model.summary()